In [1]:
# Libraries
import os
import glob
import pathlib
import time
import math
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import scipy.io

In [2]:
# Info in
matfile = 'URA3WT8_t0_dist.mat'
sts = 1
ste = 50
dts = 1 ### CHECK! Trim. By default have equal to sts.
dte = 50 ### CHECK! Trim. By default have equal to ste.

In [3]:
# Load .mat file containing cropped cells
matdat = scipy.io.loadmat(matfile)
bgi = matdat['bgimg']
crops = matdat['crops']

In [4]:
# Make output file
pathlib.Path('out').mkdir(parents=True, exist_ok=True)

In [5]:
# Set info
t = 1 # start of timecourse
ts = str(t-1).zfill(2)
z = 20 # roughly the middle of the z-stack
zs = str(z-1).zfill(2)

In [6]:
# Adjust for drift trimmed videos
if sts == dts:
    dtse = str(sts-1).zfill(2)
elif sts != dts:
    dtse = str(t + dts - 2).zfill(2)
#
tseq = []
for tss in range(sts,ste+1):
    tseq.append(tss)
tseq = np.array(tseq)
#tseq
dseq = []
for ds in range(dts,dte+1):
    dseq.append(ds)
dseq = np.array(dseq)
#dseq
# Exclude timepoints
dr = list(set(tseq) - set(dseq))
dr = np.array(dr)
dr = dr - 1
dr = dr.astype(str)
drd = []
for d in dr:
    drd.append('T'+d.zfill(2))
drd = np.array(drd)
aaa = sorted(glob.glob("../1_raw_tiff/*Z*T*.tiff"))
if len(dr) != 0:
    blacklist = re.compile('|'.join([re.escape(d) for d in drd]))
    aaa = [a for a in aaa if not blacklist.search(a)]

In [ ]:
# Loop for each cell
start1 = time.time()
clout = []
coor = []
ncells = crops.shape[0]

for num in range(ncells):

    # Cell
    #num = 4
    print('Cell: '+str(num+1).zfill(3)+'/'+str(ncells).zfill(3), end="")    
    clout.append([])
    c = int(num)

    # Get array of cell at t and z
    crctz = crops[c][0][:,:,z-1:z,t-1:t]
    crim = np.squeeze(crctz)
    cimg = crim

    # ## Can plot image of cell
    # plt.imshow(cimg, cmap=plt.get_cmap('jet'))
    # plt.show()

    # Load cell field
    azim = sorted(glob.glob("../1_raw_tiff/*Z"+zs+"*T"+dtse+".tiff"))
    aimg = mpimg.imread(azim[0])    

    # ## Can plot image of field of view
    # plt.imshow(aimg, cmap=plt.get_cmap('jet'))
    # plt.show()

    # Prepare zt-MIP
    bzim = aaa
    bza = []
    for b in bzim:
        bza.append(mpimg.imread(b))
    bza = np.array(bza)
    bztmip = np.max(bza, axis=0)

    # ## Can plot image of field of view
    # plt.imshow(bztmip*10, cmap=plt.get_cmap('viridis'))
    # plt.show()

    # Crop image shape
    csha = cimg.shape
    ccy = csha[0]
    ccx = csha[1]
    #ccx, ccy
    # Field image shape
    asha = aimg.shape
    aay = asha[0]
    aax = asha[1]
    #aax, aay

    # All possible crops of that size
    coo = []
    for i1 in range((aay+1)-ccy):
        ly = i1
        hy = i1+ccy
        for i2 in range((aax+1)-ccx):
                lx = i2
                hx = i2+ccx
                coo.append([ly,hy,lx,hx])

    # The middle of the ellipsical image crop, full width
    test = np.array(crim, copy=True)
    test[test == np.min(test)] = 0
    unv, unc = np.unique(np.where(test != 0)[0], return_counts=True)
    sec = unv[np.where(unc == np.max(unc))[0][0]]

    # Look for location in field with same intensity values as cropped image
    ROI = []
    for ce in range(len(coo)):
        taim = aimg[coo[ce][0]:coo[ce][1],coo[ce][2]:coo[ce][3]]
        if list(cimg[sec]) == list(taim[sec]):
            ROI = ce

    # If cell location is found in field
    if type(ROI) == int:
        print(' -> found')

        # Get the used pixels in elliptical crop
        tpo = np.argwhere(test != 0)
        # Find x positions of used pixels
        tpox = []
        for xt in range(coo[ROI][2], coo[ROI][3]+1):
            tpox.append(xt)
        tpox = np.array(tpox)
        # Find y positions of used pixels
        tpoy = []
        for yt in range(coo[ROI][0], coo[ROI][1]+1):
            tpoy.append(yt)
        tpoy = np.array(tpoy)

        # List of pixels
        j = []
        for jj in range(np.max(tpo[:,0])):
            j.append(jj)
        j = np.array(j)
        # Rearranged tpo
        tt = []
        for i in j:
            tt.append([])
            for ti in tpo:    
                if ti[0] == i:
                    tt[i].append(ti)

        # Used pixels as field coordinates
        fff = []
        for jj in j:
            s = tt[jj]
            s = np.array(s)
            s[:,0] = tpox[s[:,1]]
            s[:,1] = tpoy[jj]
            fff.append(s)
        fff = np.concatenate(fff)

    # If cell location isn't found in field
    if type(ROI) != int:
        print(' -> not found')
        
        fff = []
        coo = np.array([[np.float('nan'),np.float('nan'),np.float('nan'),np.float('nan')]])
        ROI = 0

    # Add to cell for loop output
    coor.append(coo[ROI])
    clout[num].append(fff)
        
end1 = time.time()
print("time:")
print(end1 - start1) 

Cell: 001/100 -> found
Cell: 002/100 -> found
Cell: 003/100 -> found
Cell: 004/100 -> found
Cell: 005/100 -> found
Cell: 006/100 -> found
Cell: 007/100 -> found
Cell: 008/100 -> found
Cell: 009/100 -> found
Cell: 010/100 -> found
Cell: 011/100 -> found
Cell: 012/100 -> found
Cell: 013/100 -> found
Cell: 014/100 -> found
Cell: 015/100 -> found
Cell: 016/100 -> found
Cell: 017/100 -> found
Cell: 018/100 -> found
Cell: 019/100 -> found
Cell: 020/100 -> found
Cell: 021/100 -> found
Cell: 022/100 -> found
Cell: 023/100 -> found
Cell: 024/100 -> found
Cell: 025/100 -> found
Cell: 026/100 -> found
Cell: 027/100 -> found
Cell: 028/100 -> found
Cell: 029/100 -> found
Cell: 030/100 -> found
Cell: 031/100 -> found
Cell: 032/100 -> found
Cell: 033/100 -> found
Cell: 034/100 -> found
Cell: 035/100 -> found
Cell: 036/100 -> found
Cell: 037/100 -> found
Cell: 038/100 -> found
Cell: 039/100 -> found
Cell: 040/100 -> found
Cell: 041/100 -> found
Cell: 042/100 -> found
Cell: 043/100 -> found
Cell: 044/1

In [ ]:
# Save cell coordinates
coordat = pd.DataFrame({
    "yl":np.array(coor)[:,0], "yh":np.array(coor)[:,1],
    "xl":np.array(coor)[:,2], "xh":np.array(coor)[:,3]
})
coordat = coordat[['xl', 'xh', 'yl', 'yh']]
coordat = coordat.reset_index()
coordat.rename(columns={'index':'cell'}, inplace=True)
coordat['cell'] = coordat['cell'] + 1
coordat.to_csv('out/cell-coord.csv', index = None)

In [ ]:
# Plot selected cell on field
plt.figure(figsize=(6,6))
plt.imshow(bztmip*10, cmap=plt.cm.viridis)
plt.xlim((0,512))
plt.ylim((512,0))
plt.savefig('out/field.png', dpi=300, bbox_inches='tight')

In [ ]:
# Plot selected cell on field
plt.figure(figsize=(6,6))
for u in range(len(clout)):
    plt.imshow(bztmip*10, cmap=plt.cm.viridis)
    plt.scatter(y=clout[u][0][:,1], x=clout[u][0][:,0], marker='s', s=1, c='white', alpha=0.05)
    plt.text(np.mean(clout[u][0][:,0]), np.mean(clout[u][0][:,1]), 
             str(u+1), fontsize=11, color='black', alpha=0.6,
             horizontalalignment='center', verticalalignment='center')
    plt.xlim((0,512))
    plt.ylim((512,0))
plt.savefig('out/field-cellpos.png', dpi=300, bbox_inches='tight')